In [ ]:
import sqlite3
import pandas as pd
import datetime
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Create your connection.
cnx = sqlite3.connect('/content/drive/MyDrive/database.sqlite')
#f_p = pd.read_sql_query("SELECT * FROM podcasts ",cnx)
#df_r = pd.read_sql_query("SELECT * FROM reviews",cnx) 
df_p = pd.read_sql_query("SELECT pod.podcast_id, pod.title,rev.author_id user_id, cat.category, pod.slug, pod.itunes_id, rev.rating, rev.content, rev.title as Review_Title, rev.created_at FROM reviews rev join podcasts pod on rev.podcast_id = pod.podcast_id join categories cat on pod.podcast_id = cat.podcast_id limit 27000", cnx)
#df_p.to_csv("/content/drive/MyDrive/Podcast_10.csv", index = True,)

In [ ]:
df_p.head(3)

Sentiment Classifier

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import string
import re
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stopword=set(stopwords.words('english'))
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
df_p["content"] = df_p["content"].apply(clean)

In [ ]:
# Initialize the SentimentIntensityAnalyzer.
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
vader = SentimentIntensityAnalyzer()

# Apply lambda function to get compound scores.
function = lambda title: vader.polarity_scores(title)['compound']
df_p['compound'] = df_p['content'].apply(function)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
data = df_p.sort_values(by=['compound'],ascending=False)
data.shape

(27000, 11)

In [ ]:
data.head()

,podcast_id,title,user_id,category,slug,itunes_id,rating,content,Review_Title,created_at,compound
13059,a3db5c2c450d4e6449de7f7e3298fb88,Inside Trader Joe's,0EFA3538EDFBA01,arts,inside-trader-joes,1375630453,5,love trader joe ♥️ love trader joe ♥️ love tra...,Love,2018-08-15T15:31:42-07:00,0.9998
13062,a3db5c2c450d4e6449de7f7e3298fb88,Inside Trader Joe's,0EFA3538EDFBA01,health-fitness-nutrition,inside-trader-joes,1375630453,5,love trader joe ♥️ love trader joe ♥️ love tra...,Love,2018-08-15T15:31:42-07:00,0.9998
13061,a3db5c2c450d4e6449de7f7e3298fb88,Inside Trader Joe's,0EFA3538EDFBA01,health-fitness,inside-trader-joes,1375630453,5,love trader joe ♥️ love trader joe ♥️ love tra...,Love,2018-08-15T15:31:42-07:00,0.9998
13060,a3db5c2c450d4e6449de7f7e3298fb88,Inside Trader Joe's,0EFA3538EDFBA01,arts-food,inside-trader-joes,1375630453,5,love trader joe ♥️ love trader joe ♥️ love tra...,Love,2018-08-15T15:31:42-07:00,0.9998
6342,f4a8f753bd3959e2a631f2eba3c98172,Pretty Basic with Alisha Marie and Remi Cruz,E9EAB9FB384204A,society-culture-personal-journals,pretty-basic-with-alisha-marie-and-remi-cruz,1439655378,5,love podcast sometim wish interven convers tal...,Love This,2019-03-08T10:23:21-07:00,0.9963


*Collaborative Filtering*

In [ ]:
items_df = df_p[['title','category','rating']]
items_df.isnull().sum()
df_sentiments = data[['title','compound']]
df_sentiments

,title,compound
13059,Inside Trader Joe's,0.9998
13062,Inside Trader Joe's,0.9998
13061,Inside Trader Joe's,0.9998
13060,Inside Trader Joe's,0.9998
6342,Pretty Basic with Alisha Marie and Remi Cruz,0.9963
...,...,...
18672,"Food for Thought: Living Compassionately, Sust...",-0.9806
14860,Monstruo,-0.9815
14859,Monstruo,-0.9815
14958,Monstruo,-0.9897


In [ ]:
from sklearn.preprocessing import LabelEncoder
lc = LabelEncoder()
data_new = data
data_new['titlen']= lc.fit_transform(data['title'])
data_new['category']= lc.fit_transform(data['category'])
data_new['user_id']= lc.fit_transform(data['user_id'])
data_new['podcast_id']= lc.fit_transform(data['podcast_id'])
items_df = data_new[['title','titlen','category','rating']]
data_new.drop(['title','slug', 'itunes_id','content','Review_Title','created_at'], axis=1, inplace=True)


In [ ]:
items_df.head(10)

,title,titlen,category,rating
13059,Inside Trader Joe's,19,0,5
13062,Inside Trader Joe's,19,14,5
13061,Inside Trader Joe's,19,13,5
13060,Inside Trader Joe's,19,4,5
6342,Pretty Basic with Alisha Marie and Remi Cruz,31,20,5
6343,Pretty Basic with Alisha Marie and Remi Cruz,31,21,5
6341,Pretty Basic with Alisha Marie and Remi Cruz,31,19,5
20404,Off Camera with Sam Jones,27,0,5
20405,Off Camera with Sam Jones,27,24,5
17850,"Food for Thought: Living Compassionately, Sust...",13,4,5


In [ ]:
data_new.head(6)

,podcast_id,user_id,category,rating,compound,titlen
13059,5,620,0,5,0.9998,19
13062,5,620,14,5,0.9998,19
13061,5,620,13,5,0.9998,19
13060,5,620,4,5,0.9998,19
6342,57,9929,20,5,0.9963,31
6343,57,9929,21,5,0.9963,31


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

lc = LabelEncoder()

data_n = df_p.drop(columns=['compound','created_at','content','slug','Review_Title','itunes_id'])
data_n['user_id'] = lc.fit_transform(df_p['user_id'])
data_n['podcast_id'] = lc.fit_transform(df_p['podcast_id'])
data_n['title'] = lc.fit_transform(df_p['title'])
data_n['category'] = lc.fit_transform(df_p['category'])

In [ ]:
items_df

In [ ]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(data_new, test_size=0.2)
train_df = train_df.reset_index(drop=True)
test_df = valid_df.reset_index(drop=True)

# Embedding

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# embedding will be initialized at random
embed = nn.Embedding(10, 3)
# given a list of ids wthe embedding corresponing to each id
a = torch.LongTensor([[1,2,0,4,5,1]])
embed(a)

tensor([[[-1.0529,  1.2599,  0.5134],
         [-2.0563, -0.3650, -0.9383],
         [-0.1725, -0.1281,  1.8099],
         [ 1.1791, -0.5794, -0.7229],
         [ 0.4474,  0.7828, -0.8423],
         [-1.0529,  1.2599,  0.5134]]], grad_fn=<EmbeddingBackward0>)


# Matrix Factroization Model

In [ ]:
class MatrixF(nn.Module):
    def __init__(self, num_users, num_pod_items, emb_size=100):
        super(MatrixF, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_pod_items, emb_size)
        self.user_emb.weight.data.uniform_(0, 0.05)
        self.item_emb.weight.data.uniform_(0, 0.05)
        
    def forward(self, u, v):
        u = self.user_emb(u)
        v = self.item_emb(v)
        return (u*v).sum(1) 

# Training Matrix Factorization Model


In [ ]:
num_users = len(train_df.user_id)
num_pod_items = len(train_df)
print(num_users, num_pod_items) 

21600 21600


Matrix Factorization using Latent feature = 20


In [ ]:
mfModel_20 = MatrixF(num_users, num_pod_items, emb_size=20)
mfModel_20

MatrixF(
  (user_emb): Embedding(21600, 20)
  (item_emb): Embedding(21600, 20)
)

In [ ]:
def train_epocs(model, epochs=10, lr=0.01, wd=0.0, unsqueeze=False):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    model.train()
    for i in range(epochs):
        users = torch.LongTensor(train_df.user_id.values) # .cuda()
        items = torch.LongTensor(train_df.podcast_id.values) #.cuda()
        ratings = torch.FloatTensor(train_df.rating.values) #.cuda()
        if unsqueeze:
            ratings = ratings.unsqueeze(1)
        y_hat = model(users, items)
        loss = F.mse_loss(y_hat, ratings)
        train_mse.append((i, loss.item()))
        if (i+1) % 10 == 0:
          print("Iteration: %d ; error = %.4f" % (i+1, loss.item()))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    test_loss(model, unsqueeze)

In [ ]:
def test_loss(model, unsqueeze=False):
    model.eval()
    users = torch.LongTensor(test_df.user_id.values) #.cuda()
    items = torch.LongTensor(test_df.podcast_id.values) #.cuda()
    ratings = torch.FloatTensor(test_df.rating.values) #.cuda()
    if unsqueeze:
        ratings = ratings.unsqueeze(1)
    y_hat = model(users, items)
    loss = F.mse_loss(y_hat, ratings)
    print("test loss %.3f " % loss.item())

In [ ]:
train_mse = []
train_epocs(mfModel_20, epochs=20, lr=0.01)

Iteration: 10 ; error = 19.1051
Iteration: 20 ; error = 13.4834
test loss 13.759 


Matrix Factorization with latent feature K= 30


In [ ]:
mfModel_30 = MatrixF(num_users, num_pod_items, emb_size=30)
train_mse = []
train_epocs(mfModel_30, epochs=20, lr=0.01)

Iteration: 10 ; error = 17.9334
Iteration: 20 ; error = 10.1785
test loss 10.710 


In [ ]:
mfModel_30 = MatrixF(num_users, num_pod_items, emb_size=30)
train_mse = []
train_epocs(mfModel_30, epochs=30, lr=0.01)

Iteration: 10 ; error = 17.9574
Iteration: 20 ; error = 10.2138
Iteration: 30 ; error = 2.1721
test loss 3.969 


Matrix Factorization with latent feature K= 70

In [ ]:
mfModel_70 = MatrixF(num_users, num_pod_items, emb_size=70)
train_mse = []
train_epocs(mfModel_70, epochs=90, lr=0.01)

Iteration: 10 ; error = 13.8901
Iteration: 20 ; error = 2.0536
Iteration: 30 ; error = 1.1984
Iteration: 40 ; error = 0.1271
Iteration: 50 ; error = 0.2330
Iteration: 60 ; error = 0.0394
Iteration: 70 ; error = 0.0487
Iteration: 80 ; error = 0.0355
Iteration: 90 ; error = 0.0285
test loss 2.343 


In [ ]:
mfModel_90 = MatrixF(num_users, num_pod_items, emb_size=90)
train_mse = []
train_epocs(mfModel_90, epochs=50, lr=0.01)

Iteration: 10 ; error = 12.0526
Iteration: 20 ; error = 0.5166
Iteration: 30 ; error = 1.3028
Iteration: 40 ; error = 0.1733
Iteration: 50 ; error = 0.1299
test loss 2.346 


In [ ]:
#predict the ratings for user 620
def predict_rating(user_id):
    user = torch.tensor([user_id])
    pod = torch.tensor(data_new['podcast_id'].unique().tolist())
    predictions = mfModel_70(user, pod).tolist()
    return predictions
predicted_r = predict_rating(9929) 
predicted_r

[5.057453155517578,
 5.008082389831543,
 5.049488067626953,
 5.038302421569824,
 5.013965606689453,
 5.006885051727295,
 4.9790544509887695,
 4.943739414215088,
 4.198768138885498,
 4.988162517547607,
 4.975846767425537,
 5.023404598236084,
 5.073990345001221,
 5.074158668518066,
 5.041748523712158,
 4.8616461753845215,
 4.989508152008057,
 5.010388374328613,
 4.876624584197998,
 5.0869035720825195,
 5.075008392333984,
 5.044510841369629,
 4.883328437805176,
 4.652141094207764,
 4.984708309173584,
 4.974112033843994,
 4.952244281768799,
 5.161203384399414,
 5.027935981750488,
 5.035090923309326,
 5.051782131195068,
 4.7596964836120605,
 4.94551944732666,
 4.877833366394043,
 4.945195198059082,
 5.153228759765625,
 4.809396266937256,
 5.17880392074585,
 4.578401565551758,
 4.735890865325928,
 5.018186569213867,
 4.99138069152832,
 5.05510139465332,
 5.032923698425293,
 4.9959635734558105,
 4.857681751251221,
 5.084360122680664,
 4.891071796417236,
 4.95535945892334,
 5.043120861053467,


In [ ]:
import numpy as np

predictions = np.array(predicted_r, dtype=np.float32)
sortedIndices = predictions.argsort()
sortedIndices

array([ 8, 38, 23, 39, 62, 31, 53, 36, 45, 15, 18, 33, 22, 47, 50,  7, 34,
       32, 55, 26, 48, 25, 10, 63,  6, 24,  9, 16, 41, 52, 44,  5,  1, 17,
        4, 60, 40, 11, 54, 28, 43, 29,  3, 14, 49, 21,  2, 30, 42,  0, 61,
       59, 51, 12, 13, 20, 58, 46, 19, 56, 57, 35, 27, 37])

In [ ]:
sortedIndices = predictions.argsort()
recommded_itemid = data_new['podcast_id'].unique()[sortedIndices][-20:][::-1]  # taking top 10
print(recommded_itemid)

[49 14 10 33 54 41 32  7 62 60 31  4 22 29  5 16  2 42  6 30]


In [ ]:
recommendedPods = items_df[items_df.titlen.isin(recommded_itemid)]
recommendedPods

,title,titlen,category,rating
6342,Pretty Basic with Alisha Marie and Remi Cruz,31,20,5
6343,Pretty Basic with Alisha Marie and Remi Cruz,31,21,5
6341,Pretty Basic with Alisha Marie and Remi Cruz,31,19,5
13714,The Leviathan Chronicles,54,12,5
13713,The Leviathan Chronicles,54,10,5
...,...,...,...,...
23818,Small Town Horror,41,5,5
23817,Small Town Horror,41,1,5
1046,Snap Judgment,42,5,2
1047,Snap Judgment,42,19,2


In [ ]:
top10k = pd.DataFrame(recommendedPods['title'])
top10k = top10k.reset_index(drop=True).drop_duplicates(keep = 'first')
#top10k = top10k.rename(columns={'index':'top 10'})
top10k

,title
0,Pretty Basic with Alisha Marie and Remi Cruz
3,The Leviathan Chronicles
13,PHOTOGRAPHY TIPS FROM THE TOP FLOOR
26,Snap Judgment
36,RABBITS
58,Wormwood: A Serialized Mystery
65,True Crime Historian
92,Bookworm
93,Major Spoilers Podcast Network Master Feed
96,Clarkesworld Magazine


Evaluation Metrics
RMSE and MAE for top 10 predcition for user 5

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error


In [ ]:
#rmse and mae

user_620_df_mf = data_new
u_nump_mf = user_620_df_mf.to_numpy()[:,2]

valid_rating_10_mf = []

for i in sortedIndices[-10:][::-1]:
  try:
    valid_rating_10_mf.append(u_nump_mf[i])
  except:
    valid_rating_10_mf.append(0)
    continue

In [ ]:
valid_rating_10_mf


[13.0, 0.0, 13.0, 0.0, 3.0, 6.0, 12.0, 0.0, 13.0, 0.0]

In [ ]:
pred_rating_10_mf = []

for i in sortedIndices[-10:][::-1]:
  try:
    pred_rating_10_mf.append(predictions[i])
  except:
    pred_rating_10_mf.append(0)
    continue

In [ ]:
len(pred_rating_10_mf),len(valid_rating_10_mf)


(10, 10)

In [ ]:
mean_squared_error(pred_rating_10_mf,valid_rating_10_mf)


35.533037391082736

In [ ]:
mean_absolute_error(pred_rating_10_mf,valid_rating_10_mf)


5.397440624237061

In [ ]:
user_5_df_mf = data_new[data_new.user_id==620]
u_nump_mf = user_5_df_mf.to_numpy()[:,2]

recommended_mf_10 = 0

for i in sortedIndices[-10:][::-1]:
  try:
    if(predictions[i] >=3):
      recommended_mf_10 +=1
      print(data_new['podcast_id'][i])
  except:
    continue

13
13
13
13
13
13
13
13
29
13


In [ ]:
recommended_mf_10

10

In [ ]:
user_5_df_mf = data_new[data_new.user_id==620]
u_nump_mf = user_5_df_mf.to_numpy()[:,2]

valid_rated_mf_10 = 10

for i in sortedIndices[-10:][::-1]:
  try:
    if(u_nump_mf[i] >=3):
      valid_rated_mf_10 +=1
      print(data_new['podcast_id'][i])
  except:
    continue

29


In [ ]:
valid_rated_mf_10

11

A hybrid recommendation system is a special type of recommendation system which can be considered as the combination of the content and collaborative filtering method. We using Neural collaborative filtering to impliment hybrid recommender system and it will build using pytorch
To build the Hybrid Recommender System, we are combining the following:
i) Movie Ratings from u.data ii) Movie genre information from u.item iii) Movie sentiment from calculated from Twitter

In [ ]:
df_p['category'].unique()

array(['arts', 'arts-performing-arts', 'music', 'arts-design',
       'education', 'society-culture', 'arts-visual-arts', 'leisure',
       'leisure-hobbies', 'arts-books', 'technology',
       'society-culture-personal-journals',
       'society-culture-relationships', 'fiction',
       'fiction-science-fiction', 'arts-food',
       'education-language-learning', 'fiction-drama', 'true-crime',
       'health-fitness', 'health-fitness-nutrition', 'history', 'comedy',
       'arts-fashion-beauty', 'tv-film'], dtype=object)

In [ ]:
items_df

In [ ]:
data_new.head(1)

,podcast_id,user_id,category,rating,compound,titlen
13059,5,620,0,5,0.9998,19


In [ ]:
dataset = df_p[['user_id','podcast_id','rating','title','category','compound']]

In [ ]:
dataset

,user_id,podcast_id,rating,title,category,compound
0,F7E5A318989779D,c61aa81c9b929a66f0c1db6cbe5d8548,5,Backstage at Tilles Center,arts,0.6908
1,F7E5A318989779D,c61aa81c9b929a66f0c1db6cbe5d8548,5,Backstage at Tilles Center,arts-performing-arts,0.6908
2,F7E5A318989779D,c61aa81c9b929a66f0c1db6cbe5d8548,5,Backstage at Tilles Center,music,0.6908
3,F6BF5472689BD12,c61aa81c9b929a66f0c1db6cbe5d8548,5,Backstage at Tilles Center,arts,0.9136
4,F6BF5472689BD12,c61aa81c9b929a66f0c1db6cbe5d8548,5,Backstage at Tilles Center,arts-performing-arts,0.9136
...,...,...,...,...,...,...
26995,00FFF40BCC67E4E,b34f7ed180cdde27a4c8ffcf2df32abe,5,B&H Photography Podcast,arts,0.6249
26996,00FFF40BCC67E4E,b34f7ed180cdde27a4c8ffcf2df32abe,5,B&H Photography Podcast,arts-visual-arts,0.6249
26997,00FFF40BCC67E4E,b34f7ed180cdde27a4c8ffcf2df32abe,5,B&H Photography Podcast,technology,0.6249
26998,B1B69F92FE6B6BE,b34f7ed180cdde27a4c8ffcf2df32abe,5,B&H Photography Podcast,arts,0.0000


In [ ]:
from sklearn.preprocessing import LabelEncoder


In [ ]:
lc = LabelEncoder()
dataset_new = dataset
dataset_new['user_id']= lc.fit_transform(dataset['user_id'])
dataset_new['podcast_id']= lc.fit_transform(dataset['podcast_id'])
dataset_new['title']= lc.fit_transform(dataset['title'])
dataset_new

<ipython-input-16-a822b3533397>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_new['user_id']= lc.fit_transform(dataset['user_id'])
<ipython-input-16-a822b3533397>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_new['podcast_id']= lc.fit_transform(dataset['podcast_id'])
<ipython-input-16-a822b3533397>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

,user_id,podcast_id,rating,title,category,compound
0,10535,29,5,3,arts,0.6908
1,10535,29,5,3,arts-performing-arts,0.6908
2,10535,29,5,3,music,0.6908
3,10481,29,5,3,arts,0.9136
4,10481,29,5,3,arts-performing-arts,0.9136
...,...,...,...,...,...,...
26995,40,16,5,1,arts,0.6249
26996,40,16,5,1,arts-visual-arts,0.6249
26997,40,16,5,1,technology,0.6249
26998,7522,16,5,1,arts,0.0000


In [ ]:
dataset_new['category']= dataset['category'].astype(int)
dataset_new['category']

In [ ]:
train_data_df,valid_data_df = train_test_split(dataset_new,test_size=0.2)


In [ ]:
train_data_df.reset_index(drop=True)
valid_data_df.reset_index(drop=True)
train_data_df

,user_id,podcast_id,rating,title,category,compound
17841,9015,38,5,13,4,0.8316
10700,4003,15,5,29,0,0.9274
25322,8841,11,5,12,3,0.0000
20703,3121,55,1,0,1,0.6369
20589,1882,49,5,35,0,0.0000
...,...,...,...,...,...,...
17737,7241,62,5,61,18,0.6369
9872,10146,18,5,38,5,0.4019
2408,3664,37,1,40,0,0.1027
25177,3966,11,5,12,0,0.6249


In [ ]:
dataset_new.shape

(27000, 6)

In [ ]:
dataset_new.isnull().sum()

user_id       0
podcast_id    0
rating        0
title         0
category      0
compound      0
dtype: int64

In [ ]:
train_data_df.head(1)

,user_id,podcast_id,rating,title,category,compound
17841,9015,38,5,13,4,0.8316


In [ ]:
num_users_hybrid = len(train_data_df.user_id)
num_pod_items_hybrid = len(train_data_df.podcast_id)
print(num_users_hybrid, num_pod_items_hybrid) 

21600 21600


In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, num_users, num_pod_items, emb_size=100, n_hidden=5):
        super(NeuralNet, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_pod_items, emb_size)
        self.layer1 = nn.Linear(emb_size*2 + n_hidden + n_hidden, n_hidden)
        self.layer2 = nn.Linear(n_hidden, 1)
        self.drop1 = nn.Dropout(0.1)
        
    def forward(self, u, v):
        user = self.user_emb(u)
        pod = self.item_emb(v[:,1])
        pod_features = v[:,4:]
        x = F.relu(torch.cat([user, pod,pod_features], dim=1))
        x = self.drop1(x)
        x = F.relu(self.layer1(x))
        x = self.layer2(x)
        return x

In [ ]:
def train_epocs_hybrid(model, epochs=10, lr=0.01, wd=0.0, unsqueeze=False):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    model.train()
    for i in range(epochs):
        users = torch.LongTensor(train_data_df.user_id.values) # .cuda()
        pods = torch.LongTensor(train_data_df.values) #.cuda()
        ratings = torch.FloatTensor(train_data_df.rating.values) #.cuda()
        if unsqueeze:
            ratings = ratings.unsqueeze(1)
        y_hat = model(users, pods)
        loss = F.mse_loss(y_hat, ratings)
        train_mse_hybrid.append((i, loss.item()))
        if (i+1) % 10 == 0:
          print("Iteration: %d ; error = %.4f" % (i+1, loss.item()))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    test_loss_hybrid(model, unsqueeze)

In [ ]:
pods = torch.LongTensor(train_data_df.values) #.cuda()
pods.shape
users = torch.LongTensor(train_data_df.user_id.values) # .cuda()
users.shape

torch.Size([21600])

In [ ]:
pods.shape

torch.Size([21600, 6])

In [ ]:
def test_loss_hybrid(model, unsqueeze=False):
    model.eval()
    users = torch.LongTensor(valid_data_df.user_id.values) #.cuda()
    pods = torch.LongTensor(valid_data_df.values) #.cuda()
    ratings = torch.FloatTensor(valid_data_df.rating.values) #.cuda()
    if unsqueeze:
        ratings = ratings.unsqueeze(1)
    y_hat = model(users, pods)
    loss = F.mse_loss(y_hat, ratings)
    print("test loss %.3f " % loss.item())

In [ ]:
users = torch.LongTensor(valid_data_df.user_id.values)

In [ ]:
pods = torch.LongTensor(valid_data_df.podcast_id.values) #.cuda()


In [ ]:
model_hybrid_20 = NeuralNet(num_users_hybrid, num_pod_items_hybrid, emb_size=20)
model_hybrid_20

NeuralNet(
  (user_emb): Embedding(21600, 20)
  (item_emb): Embedding(21600, 20)
  (layer1): Linear(in_features=50, out_features=5, bias=True)
  (layer2): Linear(in_features=5, out_features=1, bias=True)
  (drop1): Dropout(p=0.1, inplace=False)
)

In [ ]:
y_hat = model_hybrid_20(users, pods)


IndexError: ignored

In [ ]:
model_hybrid_20 = NeuralNet(num_users_hybrid, num_pod_items_hybrid, emb_size=20)
model_hybrid_20

NeuralNet(
  (user_emb): Embedding(21600, 20)
  (item_emb): Embedding(21600, 20)
  (layer1): Linear(in_features=50, out_features=5, bias=True)
  (layer2): Linear(in_features=5, out_features=1, bias=True)
  (drop1): Dropout(p=0.1, inplace=False)
)

In [ ]:
train_mse_hybrid = []
train_epocs_hybrid(model_hybrid_20, epochs=10, lr=0.01, wd=1e-6, unsqueeze=True)

RuntimeError: ignored